<h1 style="border:1px solid white; padding:5px; text-align:center; ">Tableau de bord cartographique des revenus sur GPSEA</h1>

<h2>Sujet</h2>
<div style="margin-left:20px; margin-right:20%;">
<p style="text-align:justify; margin-right:5%;">En géographie, comme dans d'autre discipline, il peut être intéressant de produire des tableaux de bord. Les tableaux de bord permettent de saisir l'information sous différents aspects, mais aussi de croiser les données dans différents graphiques. La géographie a la particularité de présenter les données spatiales sous forme de carte interactive.</p>
<p style="text-align:justify; margin-right:5%;">Le langage Python offre la possibilité de produire des tableaux de bord avec <a href="https://dash.plotly.com/">Dash</a> (reposant sur plotly.js) et d'intégrer des cartes interactives. Nativement, Dash propose les composants Scatter et ScatterMapbox, qui permettent de faire des nuages de point sur une carte. Afin de tirer parti pleinement des spécifités de la cartographie, nous utiliserons Folium (reposant sur <a href="https://leafletjs.com/">Leaflet.js</a> qui est une très puissante bibliothèque de cartographie en JavaScript).</p>
<p style="margin-right:5%;">On considère que vous avez déjà une première expérience avec Folium (production d'une carte interactive simple dans un notebook), Dash (principes des callback, des composants) et les Notebook Jupyter.</p>
<p style="margin-right:5%;"><span style="font-weight:bold;">Remarque : </span>Si le notebook n'est pas totalement lisible, vous pouvez utiliser <a href="https://nbviewer.org/">Nbviewer</a>.</p>
<p style="margin-right:5%;"><span style="font-weight:bold;">Remarque : </span>Les données de revenus sont issus de l'INSEE.</p>

</div>

<h2>Objectifs</h2>
<ul>
    <li>Préparer les données avec Pandas et Geopandas</li>
    <li>Construire le tableau de bord</li>
    <li>Construire une carte de base avec les couches par défaut</li>
    <li>Mettre à jour la carte et le graphique en même temps</li>
</ul>

<h2>Préparation des données</h2>

In [1]:
import pandas 

## revenus en 2015 France entière
revenus15 = pandas.read_csv(".\\DONNEES\\revenus_2015.csv", sep=";", decimal=".",
    dtype={"IRIS":str, "COM":str}, encoding="utf-8")

## Filtrer les données. Ne conserver que les colonnes les plus percutantes !
revenus15 = revenus15[ ["IRIS", "LIBIRIS", "COM", "LIBCOM", "DISP_MED15"] ]
## respectivement : identifiant IRIS, nom de l'IRIS, le code commune INSEE, le nom de la commune, le revenu disponible moyen en 2015
revenus15.head()

,IRIS,LIBIRIS,COM,LIBCOM,DISP_MED15
0,010040101,Les Perouses-Triangle d'Activite,01004,Ambérieu-en-Bugey,18689.0
1,010040102,Longeray-Gare,01004,Ambérieu-en-Bugey,16828.0
2,010040201,Centre-St Germain-Vareilles,01004,Ambérieu-en-Bugey,19136.0
3,010040202,Tiret-Les Allymes,01004,Ambérieu-en-Bugey,23059.0
4,010330101,Coupy-Vanchy,01033,Bellegarde-sur-Valserine,20537.0


In [2]:
## revenus en 2019 France entière
revenus19 = pandas.read_csv(".\\DONNEES\\BASE_TD_FILO_DISP_IRIS_2019.csv", sep=";", decimal=".",
    dtype={"IRIS":str}, encoding="utf-8")

revenus19 = revenus19[ ["IRIS", "DISP_MED19"] ]

revenus19.head()

,IRIS,DISP_MED19
0,10040101,19400
1,10040102,17490
2,10040201,19670
3,10040202,24310
4,10330102,20050


In [3]:
## Ensuite, on va récupérer les IRIS :
import geopandas
## Couche des communes de GPSEA
gpsea_com = geopandas.read_file(".\\DONNEES\\contours_gpsea.geojson", geometry="geometry",
                                dtype={"code_insee_commune":str}, encoding="latin1")

## La donnée est issue du catalogue de données du Grand Paris Sud Est Avenir 
gpsea_com = gpsea_com[ ["nom_de_la_commune", "code_insee_commune", "population", "geometry"] ]
## respectivement : nom de la commune, code INSEE de la commune, taille de la population, géometrie de la commune
gpsea_com.head()

gpsea_com.crs ## 4326

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

<div style="margin-left:20px; margin-right:20%;">
<p style="text-align:justify; margin-right:5%;">On rappelle que Folium utilise des coordonnées en 4326 pour afficher cartographier les données. Si vos données sont dans une projection, les passer en 4326 pour les injecter dans une carte Folium.</p>
</div>

In [4]:
gpsea_com["code_insee_commune"] = gpsea_com["code_insee_commune"].astype(str)
gpsea_com.dtypes

nom_de_la_commune       object
code_insee_commune      object
population               int64
geometry              geometry
dtype: object

In [5]:
## Couche des IRIS France entière
iris_gpsea = geopandas.read_file(".\\DONNEES\\IRIS_GE.SHP", geometry="geometry",
                                 dtype={"INSEE_COM":str, "CODE_IRIS":str})

## conserver uniquement les IRIS de GPSEA
iris_gpsea = iris_gpsea[ iris_gpsea["INSEE_COM"].isin(list(gpsea_com.code_insee_commune.unique())) ]

iris_gpsea = iris_gpsea.to_crs(4326) ## on rappel que Folium supporte uniquement les données en 4326
iris_gpsea.head()

,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS,geometry
16,94002,Alfortville,0203,940020203,Carnot Petit Pont Alouettes,H,"POLYGON ((2.41733 48.78073, 2.41803 48.78174, ..."
22,94028,Créteil,0305,940280305,Plaisance-Les Tilleuls,H,"POLYGON ((2.45363 48.79915, 2.45390 48.79927, ..."
27,94011,Bonneuil-sur-Marne,0105,940110105,Saint-Exupéry,H,"POLYGON ((2.48039 48.77052, 2.48065 48.77062, ..."
30,94044,Limeil-Brévannes,0107,940440107,Les Grands Champs,H,"POLYGON ((2.48916 48.75858, 2.48963 48.75872, ..."
35,94047,Mandres-les-Roses,0000,940470000,Mandres-les-Roses,Z,"POLYGON ((2.52681 48.70506, 2.52688 48.70510, ..."


In [6]:
revenus15 = revenus15[ revenus15["IRIS"].isin( list(iris_gpsea.CODE_IRIS.unique()) ) ]
revenus15.head()

,IRIS,LIBIRIS,COM,LIBCOM,DISP_MED15
11017,940020101,Chinagora Berthelot,94002,Alfortville,19317.0
11018,940020102,Tony Garnier Soladier,94002,Alfortville,20245.0
11019,940020103,Diderot Louis Blanc,94002,Alfortville,20522.0
11020,940020104,Marche Guesde,94002,Alfortville,20445.0
11021,940020105,Blanqui Seine Ponton,94002,Alfortville,22076.0


In [7]:
revenus19 = revenus19[ revenus19["IRIS"].isin( list(iris_gpsea.CODE_IRIS.unique()) ) ]
revenus19.head()

,IRIS,DISP_MED19
11255,940020101,21440
11256,940020102,22250
11257,940020103,22900
11258,940020104,22410
11259,940020105,24050


<div style="margin-left:20px; margin-right:20%;">
<p style="text-align:justify; margin-right:5%;">Point d'étape : nous venons de charger les données sur les revenus en 2015 et 2019 sous forme de dataframe pandas. Nous avons aussi chargé un geodataframe geopandas qui correspond aux IRIS et un autre qui correspond aux communes. Les jeux de données des revenus 2015, 2019 et les IRIS ont fait l'objet d'un filtrage pour conserver uniquement les données concernant le territoire du Grand Paris Sud Est Avenir.</p>
</div>

In [8]:
revenus_f = pandas.merge(left=revenus19, right=revenus15, left_on="IRIS", right_on="IRIS", how="left")

## On renomme surtout pour afficher les noms des colonnes dans l'application 
revenus_f.rename(columns={"DISP_MED19":"revenus medians 2019", "DISP_MED15":"revenus medians 2015"}, inplace=True)

revenus_f = revenus_f.reset_index(drop=True)
revenus_f.head()

,IRIS,revenus medians 2019,LIBIRIS,COM,LIBCOM,revenus medians 2015
0,940020101,21440,Chinagora Berthelot,94002,Alfortville,19317.0
1,940020102,22250,Tony Garnier Soladier,94002,Alfortville,20245.0
2,940020103,22900,Diderot Louis Blanc,94002,Alfortville,20522.0
3,940020104,22410,Marche Guesde,94002,Alfortville,20445.0
4,940020105,24050,Blanqui Seine Ponton,94002,Alfortville,22076.0


In [9]:
## Certains IRIS n'existaient pas en 2015. Les valeurs sont donc NA.
## Folium ne peut afficher des cercles proportionnels si une variable contient des NA. 
## On va donc remplacer le NA par des 0, de sorte à obtenir quand-même un résultat exploitable :

revenus_f["revenus medians 2015"].fillna(value=0, inplace=True)

In [10]:
## calculer le taux de variation :
import numpy 
def TauxDeVariation(row) :
    if row["revenus medians 2015"] == 0 :
        res = 0
        return res 
    elif row["revenus medians 2015"] != 0 : 
        res = ((row["revenus medians 2019"] - row["revenus medians 2015"]) / row["revenus medians 2015"]) * 100
        res = round(res, 2)
        return res 

    return res

In [11]:
import numpy 

revenus_f["tx_var_rev_med"] = numpy.nan ## taux de variation du revenu médian entre 2015 et 2019

revenus_f["tx_var_rev_med"] = revenus_f.apply(func=TauxDeVariation, axis=1)
revenus_f["tx_var_rev_med"].fillna(value=0, inplace=True)

revenus_f = pandas.merge(left=revenus_f, right=iris_gpsea[ ["CODE_IRIS", "geometry"] ], left_on="IRIS", right_on="CODE_IRIS", how="left")

In [12]:
revenus_f.isna().sum()

IRIS                     0
revenus medians 2019     0
LIBIRIS                 10
COM                     10
LIBCOM                  10
revenus medians 2015     0
tx_var_rev_med           0
CODE_IRIS                0
geometry                 0
dtype: int64

<div style="margin-left:20px; margin-right:20%;">
<p style="text-align:justify; margin-right:5%;">Point d'étape : nous avons préparé un dataframe revenus_f, qui va contenir la géométrie des IRIS ainsi que des valeurs relatives aux revenus disponibles de 2015 et 2019.</p>
</div>

## Construction d'un tableau de bord géographique interactif

In [13]:
from jupyter_dash import JupyterDash
import jupyter_dash
from dash import Dash, dcc, html, Input, Output
import dash_html_components as dchtml
import plotly.express as px 

app = JupyterDash(__name__)

C:\Users\will\AppData\Local\Temp\ipykernel_67096\949704798.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as dchtml


In [14]:
fig = px.bar(revenus_f, x="IRIS", y="revenus medians 2019", 
             color="LIBCOM", title="Barplot des revenus 2019 par IRIS")
libcom = [ com for com in list(revenus_f["LIBCOM"].unique()) ]

## https://medium.com/analytics-vidhya/valueerror-lengths-must-match-to-compare-when-adding-more-than-2-options-in-dropdown-3b4e0a5c77d4

In [15]:
## Carte interactive :
import folium 

def Create_OriginMap() :

    m = folium.Map(location=[48.7771486,2.4530731], zoom_start=12, tiles="OpenStreetMap")

    ## valeurs possibles pour style_function https://leafletjs.com/reference.html#path-option
    style_admin = {'fillColor': '#00000000', 'color': '#000000', "fill":True}    
    style_iris = {'fillColor': '#00000000', 'color': '#2b2d42', "fill":True, "weight":0.5}    

    folium.GeoJson(
        data=iris_gpsea.to_json(),
        name="Limites IRIS",
        style_function = lambda x : style_iris
    ).add_to(m).add_child(folium.features.GeoJsonPopup(fields=["NOM_COM", "CODE_IRIS"]))

    folium.GeoJson(
        data=gpsea_com.to_json(),
        name="Limites Administratives",
        style_function = lambda x : style_admin
    ).add_to(m).add_child(folium.features.GeoJsonPopup(fields=["nom_de_la_commune"]))

    folium.LayerControl().add_to(m)

    return m

In [16]:
m = Create_OriginMap()
m ## Afficher la carte interactive

In [17]:
## Créer une fenêtre IFrame pour la carte :
iframe = dchtml.Iframe(
    id="map",
    srcDoc=m._repr_html_(),
    width="90%",
    height=500
)

In [18]:
app.layout = html.Div(
    children=[
    ## Titre principal
    html.H1("Application cartographique interactive sur les revenus à GPSEA",
            style={"textAlign":"center"}),
            
            ## container pour le barplot :
            html.Div(style={"border":"solid black 1px"}, children=[
    html.H2("Barplot des revenus médians dans les IRIS de GPSEA"),
    
    dcc.Dropdown(options=libcom, value=libcom[0], id="choix_com", multi=True),

    dcc.RadioItems(id="radio_items_revenus", options=[ "revenus medians 2015", "revenus medians 2019"], value="revenus medians 2019"),

    dcc.Graph(id="graph",
              figure=fig)
              ]),

              ## container pour la carte interactive folium :
              html.Div(style={"border":"solid red 1px"}, children=[
    html.H2("Carte interactive"),
    iframe
              ])
              ])

In [19]:
def Create_Prop_Circles(df, annee) :
    m = Create_OriginMap() ## créer une carte interactive

    def TailleCercle(row) :
        res = numpy.sqrt(row[annee])

        return res
    
    ## Calculer 
    df["diam"] = df.apply(func=TailleCercle, axis=1)

    ## Créer des cercles proportionnels 
    for i, r in df.iterrows() :
        folium.Circle(
            radius=r["diam"],
            location=[r.geometry.centroid.y, r.geometry.centroid.x],
            stroke=True, ## bordure 
            color="#219ebc", ## couleur de la bordure 
            weight=0.5, ## taille de la bordure
            opacity=0.5, ## opacité (préférer une certaine transparence pour la lisbilité)
            fill=True, ## Activer le remplissage
            fillOpacity=0.5, ## opacité du remplissage
            fillcolor="#8ecae6", 
            popup= f"{r['LIBIRIS']} ({r['IRIS']}) \n revenus médians {r[annee]}"## Afficher du texte
        ).add_to(m)

    return m._repr_html_()

In [20]:
## Les callbacks :

## Mettre à jour le barplot 
@app.callback(
    [Output("graph", "figure"),
     Output("map", "srcDoc")],
    [Input("choix_com", "value"), ## choix_de_commune
    Input("radio_items_revenus", "value")] ## chx_annee
)
def update_figure(choix_de_commune, chx_annee) :
    if type(choix_de_commune) != str :
        res = revenus_f[ (revenus_f["LIBCOM"].isin(choix_de_commune)) ]

    else :
        res = revenus_f[ revenus_f["LIBCOM"] == choix_de_commune ]

    fig = px.bar(res, x="IRIS", y=chx_annee, color="LIBCOM", title=f"Barplot des {chx_annee} par IRIS")

    fig.update_layout(transition_duration=500)

    ## Mettre à jour la carte :
    m = Create_Prop_Circles(res, chx_annee)

    return fig, m

In [21]:
if __name__ == "__main__" :
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


C:\Users\will\AppData\Local\Temp\ipykernel_67096\2290041771.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\will\AppData\Local\Temp\ipykernel_67096\2290041771.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\will\AppData\Local\Temp\ipykernel_67096\2290041771.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda